'''
This program selects future-oriented text and cleans text
#Date: Dec 3 2019
#Update: Mar 25, 2019
#Author: Carly Knight
'''

### Import packages

In [41]:
import os
from zipfile import ZipFile
import xml.etree.ElementTree as ET
from xml import etree
import csv
import re
import dateutil.parser as parser
from enchant.tokenize import get_tokenizer
import enchant
import pandas as pd
from itertools import chain
import random

## get liwc list


In [42]:
liwclist= "/Users/carlyknight/Dropbox/PROJECTS/ConceptionsofRisk/data/LIWC/liwc2015dict.csv"
lw= pd.read_csv(liwclist)

#select FocusFuture
future= lw.loc[lw['var'] == 'FocusFuture']
futurelist = future['word'].tolist()

#stems versus words
futurelist_stem = [s.strip('*') for s in futurelist if "*" in s]
futurelist_word = [s for s in futurelist if "*" not in s]

#add/remove
futurelist_word.extend(["Future", "will", "is heading", "are heading", "aim","could", "doubt", "has begun", "intend", "intends", "intending", "is expected", "is likely to", "it is hoped", "optimism", "pessimism", "uncertainty", "projected", "certainty", "as yet", "not yet", "may", "outlook", "should", "risk", "risky", "risk-taking", "is planned", "is planning", "is scheduled", "a planned", "the planned", "optimistic", "pessimistic", "next generation", "well positioned", "next-generation", "well-positioned", "is scheduled", "are scheduled"])
futurelist_word.remove("plan")
futurelist_word.remove("plans")
futurelist_word.remove("planner")
futurelist_word.remove("planning")

futurelist_stem.extend(["confident"])
futurelist_stem.remove("headin")
futurelist_stem.remove("prepar")

#notwordlist
notwordlist=["shareholders will", "Shareholders will", "will be of interest", "good will", "GOOD WILL", "Good will", "you will", "was expected", "were expected", "TABLE OF CONTENTS", "table fo contents", "should have", "could have", "would have", "should not have", "could not have", "would not have", "may have"]

#test examples
example_list = ["possible"]
example_container_lists = [ [] for i in range(len(example_list)) ]

print(futurelist_word)
print(futurelist_stem)
print(notwordlist)
#Other potential options:
#Risk, Money, Quant, Verb, Tentat, Certain, Time

['ahead', 'anticipation', 'approaching', 'attainable', 'coming', 'eventual', 'eventually', 'fate', 'fated', 'fates', 'feasible', 'finna', 'fixin', "fixin'", 'foresight', 'forseeable', 'forthcoming', 'going', 'gon', 'gonna', 'gotta', 'gunna', "he'll", 'henceforth', 'hope', 'hopeful', 'hopefully', 'hoping', "i'll", "i'mma", 'ima', 'imma', 'imminent', 'impending', "it'll", 'itll', 'looming', 'may', 'might', 'obtainable', 'oncoming', 'onward', 'pray', 'prayed', 'praying', 'promising', 'shall', "shan't", 'shant', "she'll", 'someday', 'sometime', 'soon', 'sooner', 'soonest', "that'll", 'thatll', 'Future', 'will', 'is heading', 'are heading', 'aim', 'could', 'doubt', 'has begun', 'intend', 'intends', 'intending', 'is expected', 'is likely to', 'it is hoped', 'optimism', 'pessimism', 'uncertainty', 'projected', 'certainty', 'as yet', 'not yet', 'may', 'outlook', 'should', 'risk', 'risky', 'risk-taking', 'is planned', 'is planning', 'is scheduled', 'a planned', 'the planned', 'optimistic', 'pes

# Prep + helper functions

### Input and Output Data Location

In [1]:
directory = "/Users/carlyknight/Documents/Data/Annual Report/XML/"
results= "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts/"
test_files= "/Users/carlyknight/Dropbox/PROJECTS/ConceptionsofRisk/data/keyword_test_files/"
test_paragraphs = "/Users/carlyknight/Dropbox/PROJECTS/Corporate_Futures_for_Hesu/paragraphs/"

### Help function: spell checker

In [44]:
#function to calculate the percent
def calculate_percent_correct(text):
	true =0
	total = 0.0
	for word in tknzr(text):
		word1 =word[0]
		total +=1
		if d2.check(word1):
			true +=1
	if total !=0:
		percent = true*1.0/total
	else:
		percent =0
	return percent

'''INITIATE SPELLCHECKER'''
d2 = enchant.DictWithPWL("en_US")
tknzr = get_tokenizer("en_US")

### Helper function: clean text

In [45]:
def get_fulltext(root):
	full = root.find('FullText')
	if full is None:
		fulltext = ""
	else:
		fulltext=full.text
	return fulltext

In [46]:
def clean_punction(fulltext):
    #remove various kinds of superfluous punction
    fulltext = re.sub(r'[;:$-*><=|^·\,\\-]','',fulltext)
    fulltext.replace(u'No.', u'No')
    fulltext.replace("\\", "") 
    #replace series of periods with one period
    fulltext = re.sub(r'(\.\s*\.\s*)+', '', fulltext)
    return fulltext

### Regex Compile

In [47]:
pattern_word = r"(?<=\W)(%s)\b" % "|".join(futurelist_word)
regex_word = re.compile(pattern_word)

pattern_stem = r"(?<=\W)(%s)" % "|".join(futurelist_stem)
regex_stem = re.compile(pattern_stem)


### Helper fuction: Wordsearch functions

In [48]:
#identify paragraphs
def paragraph_split(fulltext):
    splits= re.split('(\.{3,}[-_\s]*\.{2,}[-_\s]*)|(\_{3,})|(\-{3,})|(\s{3,}[0-9]{1,}\s{3,})|(\s{5,}[A-Z\s]+\s{5,})', fulltext)
    splits=[split for split in splits if split is not None if split.strip() is not '']
    return splits

#combine paragraphs if they are too short
def paragraph_combine(si, length):
    si = iter(si)
    while True:
        try:
            current = next(si)
        except StopIteration:
            return
        try:
            while len(current) < length:
                current += ' ' + next(si)
        except StopIteration:
            return
        try:
            yield current
        except StopIteration:
            return
        

In [49]:
def paragraph_wordsearch(paragraphs):
    selected_paragraphs = []
    for paragraph in paragraphs:
        #search paragraph for wordlist
        num_matches=len(regex_stem.findall(paragraph)) + len(regex_word.findall(paragraph))
        if num_matches > 0: #if there is a match
            #chain the word and stem iterators:
            chain_iter = chain(regex_stem.finditer(paragraph), regex_word.finditer(paragraph))
            #iterate through chain
            for match in chain_iter:
                #make sure only match is not in not-match list:
                if [m for m in notwordlist if m in paragraph[match.start()-10: match.end()+10]]:
                    continue
                else:
                    #replace with asterisks
                    paragraph =paragraph.replace(match.group(), "***" + match.group())
            #substitute due to loop problems
            paragraph = re.sub(r'\*+', "***", paragraph)
            #if there are asterisks, then append
            if "***" in paragraph:
                selected_paragraphs.append(paragraph)
    return selected_paragraphs


In [50]:
#identify only paragraphs with a given data quality
def select_good_paragraphs(paragraphs):
    selected_paragraphs = []
    for paragraph in paragraphs:
        #remove paragraphs that begin with "Note"
        if paragraph.split()[0] == "Note" or paragraph.split()[0] == "NOTE":
            continue
        # keep only paragraphs that aren't just numbers 
        nums=sum(c.isdigit() for c in paragraph)
        percent_digits = (nums*1.0)/len(paragraph)
        #keep sections with spell correctness above .85
        spellcorrect= calculate_percent_correct(paragraph)
        #remove sections with list of all capitalized words (most likely to be names)
        percent_uppers = (1.0*sum(1 for c in paragraph if c.isupper()))/len(paragraph)
        if percent_digits <.12 and percent_uppers <.5 and spellcorrect >.85 :
            selected_paragraphs.append(paragraph)
    return selected_paragraphs

In [51]:
def paragraph_example_wordsearch(selected_text):
    paragraphs = selected_text.split("####")
    for paragraph in paragraphs: #iterate through paragraph
        for i,item in enumerate(example_list): #for each paragraph, test for each item
            if re.search(r"(?<=\*\*\*)" + item + r"\b", paragraph):
                example_container_lists[i].append(paragraph)

### Helper function: get metadata

In [52]:
def get_metadata(root):
    date= root.find('AlphaPubDate').text
    datenum = root.find('NumericPubDate').text
    year = parser.parse(date).year
    url = root.find('URLDocView').text
    child = root.find('Terms')
    #create empty containers for terms
    company = ''
    naics=''
    code = ''
    classic = ''
    #iterate through terms
    for alltags in child.findall('.//'):
        if alltags.tag == "CompanyName":
            company = alltags.text
        elif alltags.tag == "CompanyNAIC":
            naics = alltags.text
        elif alltags.tag == "ClassCode":
            code = alltags.text
        elif alltags.tag == "ClassExpansion":
            classic = alltags.text
    # put the flexterms into a dictionary
    dict = {}
    for elem in root.iter('FlexTerm'):
        dict[elem.find('FlexTermName').text] = elem.find('FlexTermValue').text
    # create terms from dictionary
    ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns = [dict.get(k) for k in
                                                                            ['AnrClassCode1IdxLit',
                                                                             'AnrCashDispIdxNum',
                                                                             'AnrCashSrchIdxNum',
                                                                             'AnrAssetDispIdxNum',
                                                                             'AnrAssetSrchIdxNum',
                                                                             'AnrRevDispIdxNum',
                                                                             'AnrRevSrchIdxNum',
                                                                             'AnrEarnDispIdxNum',
                                                                             'AnrEarnSrchIdxNum']]
    # if variables above are NONE, recode as empty string
    ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns = [k or "" for k in
                                                                            [ancode, ancashd, ancashs, anasd,
                                                                             anass, anrevd, anrevs, anearnd,
                                                                             anearns]]
    #create list of alternative company names
    altnames =[]
    for elem in root.iter('FlexTerm'):
        if elem.find('FlexTermName').text == "AnrRelCoNameIdxLit":
            altnames.append(elem.find('FlexTermValue').text)
    relnames='; '.join(altnames)
    return date, datenum, year, url, company, naics, code, classic, ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns, relnames
    

# Main loop

### create list for random selection to check

In [53]:
random_selection = []

### create list of zip-files

In [54]:
zippaths=[]
for zipped in os.listdir(directory):
	if not zipped.endswith('.zip'):continue
	zippaths.append(os.path.join(directory, zipped))


### open csv for metadata

In [57]:
csvfile = open(results + 'metadata_futureperfect.csv', 'w')
fieldnames = ['Location', 'Filename', 'RecordTitle', 'Date', 'Date1', 'Year', 'URL', 'CompanyName', 'NAIC', 'ClassCode', 'Classification', 'AnrAssetDispIdxNum', 'numwords', 'num_paragraphs_prespellcheck', 'num_paragraphs_postspellcheck']
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
writer.writeheader()
            

# Loop

In [58]:
i=1
for filename in zippaths:
    with ZipFile(filename, "r") as zf:
        for name in zf.namelist():
            data= zf.read(name)
            root = ET.fromstring(data)
            date, datenum, year, url, company, naics, code, classic, ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns, relnames= get_metadata(root)
            #get fulltext
            fulltext= get_fulltext(root)
            #clean fulltext
            fulltext=clean_punction(fulltext)
            #identify paragraphs
            paragraphs1 = paragraph_split(fulltext)
            paragraphs2 = list(paragraph_combine(paragraphs1, 200))
            #wordsearch paragraphs
            paragraphs3= paragraph_wordsearch(paragraphs2)
            n_paragraphs_pre= len(paragraphs3)
            #select only quality paragraphs
            paragraphs4 = select_good_paragraphs(paragraphs3)
            if paragraphs4 is None or len(paragraphs4)==0:
                print(str(i), name, "No results returned")
                continue
            n_paragraphs_post= len(paragraphs4)
            #select a random paragraph
            #selection = random.choice(paragraphs4)
            #random_selection.append([name, selection])
            #populate example lists
            #paragraph_example_wordsearch(selected_text)
            #calculate percent correct
            #percentcorrect = calculate_percent_correct(selected_text)
            #join text
            selected_text= "####".join(paragraphs4)
            #calculate number of words
            numwords = len(selected_text.split())
            #put results in a dict
            resultsdict= {'Filename': name, 'RecordTitle': id, 'Date': date, "Date1": datenum, 'Year': year, 'URL': url, 'CompanyName': company.encode('utf-8'), 'NAIC': naics, 'ClassCode': code, 'Classification': classic.encode('utf-8'), 'AnrAssetDispIdxNum': anasd, 'numwords': numwords, 'num_paragraphs_prespellcheck': n_paragraphs_pre, 'num_paragraphs_postspellcheck': n_paragraphs_post}
            #write to metadata file
            writer.writerow(resultsdict)
            #write out selected text
            with open(results + re.sub(".xml", ".txt", name), "w") as text_file:
                _=text_file.write(selected_text)
            print(str(i), name, "Written")
            i+=1
            
            

1 88201676.xml Written
2 88179860.xml Written
3 88198679.xml Written
4 88190194.xml Written
5 88178029.xml Written
6 88198719.xml Written
7 88177949.xml Written
8 88180180.xml Written
9 88205316.xml Written
10 88180349.xml Written
11 88203156.xml Written
12 88179900.xml Written
13 88179100.xml Written
14 88178669.xml No results returned
14 88175669.xml Written
15 88197156.xml Written
16 88177269.xml No results returned
16 88201519.xml Written
17 88193154.xml Written
18 88200119.xml Written
19 88179620.xml Written
20 88176989.xml No results returned
20 88178229.xml Written
21 88191354.xml Written
22 88179420.xml Written
23 88197999.xml Written
24 88176869.xml Written
25 88189354.xml Written
26 88175429.xml Written
27 88204516.xml Written
28 88178909.xml Written
29 88173700.xml Written
30 88180420.xml No results returned
30 88177220.xml Written
31 88176909.xml Written
32 88174700.xml Written
33 88180900.xml Written
34 88175509.xml Written
35 88178069.xml Written
36 88180389.xml Written
3

289 88191834.xml Written
290 88172895.xml No results returned
290 88214451.xml Written
291 88213811.xml Written
292 88193074.xml Written
293 88208091.xml Written
294 88183627.xml Written
295 88172699.xml Written
296 88183267.xml Written
297 88174055.xml Written
298 88205371.xml Written
299 88181268.xml Written
300 88201451.xml Written
301 88208105.xml Written
302 88205942.xml Written
303 88208185.xml Written
304 88175868.xml Written
305 88177308.xml No results returned
305 88173335.xml No results returned
305 88199980.xml Written
306 88181307.xml Written
307 88190634.xml Written
308 88179387.xml No results returned
308 88182788.xml Written
309 88205611.xml Written
310 88191714.xml Written
311 88176588.xml Written
312 88191394.xml Written
313 88172977.xml Written
314 88176948.xml Written
315 88207785.xml Written
316 88185707.xml Written
317 88182588.xml Written
318 88173497.xml No results returned
318 88174698.xml Written
319 88175932.xml Written
320 88177867.xml Written
321 88192045.xm

568 88224356.xml Written
569 88225916.xml Written
570 88216596.xml Written
571 88176912.xml Written
572 88206116.xml Written
573 88223676.xml Written
574 88207156.xml Written
575 88178632.xml Written
576 88207076.xml Written
577 88218116.xml Written
578 88208236.xml Written
579 88223996.xml Written
580 88207756.xml Written
581 88223116.xml Written
582 88220916.xml Written
583 88222116.xml Written
584 88228836.xml Written
585 88181913.xml Written
586 88228803.xml Written
587 88181393.xml Written
588 88223756.xml Written
589 88180352.xml Written
590 88180032.xml Written
591 88228916.xml Written
592 88212916.xml Written
593 88223316.xml Written
594 88181193.xml Written
595 88180913.xml Written
596 88206516.xml Written
597 88208476.xml Written
598 88206076.xml Written
599 88223569.xml Written
600 88176872.xml No results returned
600 88180832.xml Written
601 88209196.xml Written
602 88223969.xml Written
603 88176552.xml No results returned
603 88207716.xml Written
604 88206196.xml Written
6

851 88204115.xml Written
852 88197392.xml Written
853 88195643.xml Written
854 88203192.xml Written
855 88189518.xml Written
856 88189798.xml Written
857 88199312.xml Written
858 88187005.xml Written
859 88231164.xml Written
860 88204194.xml Written
861 88180457.xml Written
862 88177777.xml No results returned
862 88176897.xml Written
863 88180174.xml Written
864 88202438.xml Written
865 88190758.xml Written
866 88214233.xml Written
867 88186678.xml Written
868 88192963.xml Written
869 88197123.xml Written
870 88198598.xml Written
871 88192118.xml Written
872 88198672.xml Written
873 88193523.xml Written
874 88188099.xml Written
875 88187958.xml Written
876 88220553.xml Written
877 88183979.xml No results returned
877 88206152.xml Written
878 88205352.xml Written
879 88213433.xml Written
880 88221004.xml Written
881 88191878.xml Written
882 88182174.xml No results returned
882 88197312.xml Written
883 88225547.xml Written
884 88214032.xml Written
885 88230232.xml Written
886 88229435.x

1141 88174744.xml Written
1142 88175865.xml Written
1143 88178304.xml No results returned
1143 88178384.xml Written
1144 88175225.xml Written
1145 88177745.xml No results returned
1145 88174584.xml Written
1146 88173780.xml Written
1147 88174820.xml No results returned
1147 88176220.xml Written
1148 88176345.xml Written
1149 88174300.xml Written
1150 88176465.xml No results returned
1150 88175145.xml Written
1151 88175466.xml No results returned
1151 88174824.xml No results returned
1151 88176705.xml No results returned
1151 88176062.xml No results returned
1151 88175224.xml Written
1152 88173620.xml Written
1153 88176624.xml Written
1154 88175265.xml Written
1155 88175186.xml Written
1156 88176184.xml Written
1157 88177585.xml No results returned
1157 88174985.xml No results returned
1157 88199055.xml Written
1158 88201375.xml Written
1159 88199530.xml Written
1160 88182081.xml Written
1161 88184841.xml Written
1162 88183187.xml Written
1163 88179948.xml No results returned
1163 88207

...

39710 88200507.xml Written
39711 88203309.xml Written
39712 88202290.xml Written
39713 88184161.xml Written
39714 88198170.xml Written
39715 88179385.xml Written
39716 88203827.xml Written
39717 88177721.xml Written
39718 88198695.xml Written
39719 88191582.xml Written
39720 88195486.xml Written
39721 88181481.xml Written
39722 88206020.xml Written
39723 88179625.xml Written
39724 88205343.xml Written
39725 88187902.xml Written
39726 88204389.xml Written
39727 88214149.xml Written
39728 88183681.xml Written
39729 88179281.xml Written
39730 88178201.xml Written
39731 88198615.xml Written
39732 88189606.xml Written
39733 88201450.xml Written
39734 88188902.xml Written
39735 88192662.xml Written
39736 88201775.xml Written
39737 88178265.xml Written
39738 88177201.xml Written
39739 88208389.xml Written
39740 88209349.xml Written
39741 88205109.xml Written
39742 88177041.xml Written
39743 88199867.xml Written
39744 88194575.xml Written
39745 88196770.xml Written
39746 88173241.xml Written
3

40007 88202380.xml Written
40008 88195260.xml Written
40009 88196840.xml Written
40010 88200060.xml Written
40011 88197360.xml Written
40012 88193045.xml Written
40013 88199300.xml Written
40014 88199085.xml Written
40015 88191222.xml Written
40016 88191585.xml Written
40017 88196080.xml Written
40018 88195925.xml Written
40019 88198625.xml Written
40020 88192820.xml Written
40021 88197020.xml Written
40022 88187542.xml Written
40023 88188142.xml Written
40024 88201285.xml Written
40025 88196505.xml Written
40026 88191700.xml Written
40027 88194565.xml Written
40028 88201225.xml Written
40029 88201905.xml Written
40030 88223929.xml Written
40031 88201747.xml Written
40032 88230206.xml Written
40033 88212683.xml Written
40034 88229988.xml Written
40035 88227120.xml Written
40036 88203669.xml Written
40037 88231483.xml Written
40038 88197627.xml Written
40039 88196762.xml Written
40040 88193122.xml Written
40041 88203147.xml Written
40042 88222643.xml Written
40043 88220923.xml Written
4

40296 88225323.xml Written
40297 88175604.xml Written
40298 88179089.xml Written
40299 88229565.xml Written
40300 88175684.xml Written
40301 88179880.xml No results returned
40301 88224249.xml Written
40302 88229169.xml Written
40303 88232282.xml Written
40304 88222883.xml Written
40305 88175561.xml No results returned
40305 88177761.xml Written
40306 88201136.xml Written
40307 88180281.xml Written
40308 88197016.xml Written
40309 88179821.xml No results returned
40309 88189766.xml Written
40310 88189886.xml Written
40311 88184480.xml Written
40312 88177361.xml Written
40313 88192086.xml Written
40314 88190505.xml Written
40315 88177383.xml Written
40316 88195086.xml Written
40317 88192340.xml Written
40318 88213524.xml Written
40319 88181743.xml Written
40320 88188654.xml Written
40321 88202859.xml Written
40322 88184721.xml No results returned
40322 88229084.xml Written
40323 88179881.xml No results returned
40323 88189526.xml Written
40324 88183640.xml Written
40325 88222546.xml Wri

40572 88199299.xml Written
40573 88203456.xml Written
40574 88183501.xml Written
40575 88193488.xml Written
40576 88201539.xml Written
40577 88184325.xml No results returned
40577 88187125.xml Written
40578 88204376.xml Written
40579 88184748.xml No results returned
40579 88230987.xml Written
40580 88195100.xml Written
40581 88186968.xml Written
40582 88192561.xml Written
40583 88193626.xml Written
40584 88193803.xml Written
40585 88192420.xml Written
40586 88193180.xml Written
40587 88189305.xml Written
40588 88194203.xml Written
40589 88181701.xml Written
40590 88184005.xml Written
40591 88185440.xml Written
40592 88203896.xml Written
40593 88200579.xml Written
40594 88185468.xml Written
40595 88191361.xml Written
40596 88194585.xml Written
40597 88192060.xml Written
40598 88200539.xml Written
40599 88191500.xml Written
40600 88193066.xml Written
40601 88202299.xml Written
40602 88185640.xml Written
40603 88189066.xml Written
40604 88197339.xml Written
40605 88195380.xml Written
4060

40867 88205507.xml Written
40868 88204340.xml Written
40869 88207903.xml Written
40870 88182721.xml Written
40871 88188502.xml Written
40872 88195006.xml Written
40873 88194695.xml Written
40874 88192502.xml Written
40875 88195015.xml Written
40876 88194090.xml Written


## Create text file with examples

In [192]:
#for i,item in enumerate(example_list):
#    with open(test_files +  item.replace(" ", "") + '2.txt', 'w') as f:
#        f.write('%s\n\n' % example_container_lists[i])

## Create selection of 1000 random paragrahs

In [211]:
#how many were selected?
len(random_selection)

38060

In [212]:
#select 1000 random paragraphs 
random_sample = random.choices(random_selection, k=1000)
len(random_sample)

1000

In [213]:
#put these in a csv
with open(test_paragraphs + "random_sample_v1.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(random_sample)

In [214]:
paragraphs4

['Shareowners are cordially invited to attend the Annual Meeting of the company at The Greenbner White Sulphur Springs W. Va. on March 24 1961. Details ***will be given in the Notice of the Annual MeetingBeginning Another Great Era of Growth']